# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.org/).

## Setting up libCEED for Python

Install libCEED for Python by running

In [ ]:
! python -m pip install libceed

## CeedBasis

Here we show some basic examples to illustrate the `libceed.Basis` class. In libCEED, a `libceed.Basis` defines the finite element basis and associated quadrature rule (see [the API documentation](https://libceed.org/en/latest/libCEEDapi.html#finite-element-operator-decomposition)).

First we declare some auxiliary functions needed in the following examples

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def eval(dim, x):
  result, center = 1, 0.1
  for d in range(dim):
    result *= np.tanh(x[d] - center)
    center += 0.1
  return result

def feval(x1, x2):
  return x1*x1 + x2*x2 + x1*x2 + 1

def dfeval(x1, x2):
  return 2*x1 + x2

## $H^1$ Lagrange bases in 1D

The Lagrange interpolation nodes are at the Gauss-Lobatto points, so interpolation to Gauss-Lobatto quadrature points is the identity.

In [ ]:
import libceed

ceed = libceed.Ceed()

b = ceed.BasisTensorH1Lagrange(
    dim=1,   # topological dimension
    ncomp=1, # number of components
    P=4,     # number of basis functions (nodes) per dimension
    Q=4,     # number of quadrature points per dimension
    qmode=libceed.GAUSS_LOBATTO)
print(b)

Although a `libceed.Basis` is fully discrete, we can use the Lagrange construction to extend the basis to continuous functions by applying `EVAL_INTERP` to the identity.  This is the Vandermonde matrix of the continuous basis.

In [ ]:
P = b.get_num_nodes()
nviz = 50
bviz = ceed.BasisTensorH1Lagrange(1, 1, P, nviz, libceed.GAUSS_LOBATTO)

# Construct P "elements" with one node activated
I = ceed.Vector(P * P)
with I.array(P, P) as x:
    x[...] = np.eye(P)

Bvander = ceed.Vector(P * nviz)
bviz.apply(4, libceed.EVAL_INTERP, I, Bvander)

qviz, _weight = ceed.lobatto_quadrature(nviz)
with Bvander.array_read(nviz, P) as B:
    plt.plot(qviz, B)

# Mark tho Lobatto nodes
qb, _weight = ceed.lobatto_quadrature(P)
plt.plot(qb, 0*qb, 'ok');

In contrast, the Gauss quadrature points are not collocated, and thus all basis functions are generally nonzero at every quadrature point.

In [ ]:
b = ceed.BasisTensorH1Lagrange(1, 1, 4, 4, libceed.GAUSS)
print(b)

with Bvander.array_read(nviz, P) as B:
    plt.plot(qviz, B)
# Mark tho Gauss quadrature points
qb, _weight = ceed.gauss_quadrature(P)
plt.plot(qb, 0*qb, 'ok');

Although the underlying functions are not an intrinsic property of a `libceed.Basis` in libCEED, the sizes are.
Here, we create a 3D tensor product element with more quadrature points than Lagrange interpolation nodes.

In [ ]:
b = ceed.BasisTensorH1Lagrange(3, 1, 4, 5, libceed.GAUSS_LOBATTO)

p = b.get_num_nodes()
print('p =', p)

q = b.get_num_quadrature_points()
print('q =', q)

* In the following example, we demonstrate the application of an interpolatory basis in multiple dimensions

In [ ]:
for dim in range(1, 4):
  Q = 4
  Qdim = Q**dim
  Xdim = 2**dim
  x = np.empty(Xdim*dim, dtype="float64")
  uq = np.empty(Qdim, dtype="float64")

  for d in range(dim):
    for i in range(Xdim):
      x[d*Xdim + i] = 1 if (i % (2**(dim-d))) // (2**(dim-d-1)) else -1

  X = ceed.Vector(Xdim*dim)
  X.set_array(x, cmode=libceed.USE_POINTER)
  Xq = ceed.Vector(Qdim*dim)
  Xq.set_value(0)
  U = ceed.Vector(Qdim)
  U.set_value(0)
  Uq = ceed.Vector(Qdim)

  bxl = ceed.BasisTensorH1Lagrange(dim, dim, 2, Q, libceed.GAUSS_LOBATTO)
  bul = ceed.BasisTensorH1Lagrange(dim, 1, Q, Q, libceed.GAUSS_LOBATTO)

  bxl.apply(1, libceed.EVAL_INTERP, X, Xq)

  with Xq.array_read() as xq:
    for i in range(Qdim):
      xx = np.empty(dim, dtype="float64")
      for d in range(dim):
        xx[d] = xq[d*Qdim + i]
      uq[i] = eval(dim, xx)

  Uq.set_array(uq, cmode=libceed.USE_POINTER)

  # This operation is the identity because the quadrature is collocated
  bul.T.apply(1, libceed.EVAL_INTERP, Uq, U)

  bxg = ceed.BasisTensorH1Lagrange(dim, dim, 2, Q, libceed.GAUSS)
  bug = ceed.BasisTensorH1Lagrange(dim, 1, Q, Q, libceed.GAUSS)

  bxg.apply(1, libceed.EVAL_INTERP, X, Xq)
  bug.apply(1, libceed.EVAL_INTERP, U, Uq)

  with Xq.array_read() as xq, Uq.array_read() as u:
    #print('xq =', xq)
    #print('u =', u)
    if dim == 2:
        # Default ordering is contiguous in x direction, but
        # pyplot expects meshgrid convention, which is transposed.
        x, y = xq.reshape(2, Q, Q).transpose(0, 2, 1)
        plt.scatter(x, y, c=np.array(u).reshape(Q, Q))
        plt.xlim(-1, 1)
        plt.ylim(-1, 1)
        plt.colorbar(label='u')

* In the following example, we demonstrate the application of the gradient of the shape functions in multiple dimensions

In [ ]:
for dim in range (1, 4):
  P, Q = 8, 10
  Pdim = P**dim
  Qdim = Q**dim
  Xdim = 2**dim
  sum1 = sum2 = 0
  x = np.empty(Xdim*dim, dtype="float64")
  u = np.empty(Pdim, dtype="float64")

  for d in range(dim):
    for i in range(Xdim):
      x[d*Xdim + i] = 1 if (i % (2**(dim-d))) // (2**(dim-d-1)) else -1

  X = ceed.Vector(Xdim*dim)
  X.set_array(x, cmode=libceed.USE_POINTER)
  Xq = ceed.Vector(Pdim*dim)
  Xq.set_value(0)
  U = ceed.Vector(Pdim)
  Uq = ceed.Vector(Qdim*dim)
  Uq.set_value(0)
  Ones = ceed.Vector(Qdim*dim)
  Ones.set_value(1)
  Gtposeones = ceed.Vector(Pdim)
  Gtposeones.set_value(0)

  # Get function values at quadrature points
  bxl = ceed.BasisTensorH1Lagrange(dim, dim, 2, P, libceed.GAUSS_LOBATTO)
  bxl.apply(1, libceed.EVAL_INTERP, X, Xq)

  with Xq.array_read() as xq:
    for i in range(Pdim):
      xx = np.empty(dim, dtype="float64")
      for d in range(dim):
        xx[d] = xq[d*Pdim + i]
      u[i] = eval(dim, xx)

  U.set_array(u, cmode=libceed.USE_POINTER)

  # Calculate G u at quadrature points, G' * 1 at dofs
  bug = ceed.BasisTensorH1Lagrange(dim, 1, P, Q, libceed.GAUSS)
  bug.apply(1, libceed.EVAL_GRAD, U, Uq)
  bug.T.apply(1, libceed.EVAL_GRAD, Ones, Gtposeones)

  # Check if 1' * G * u = u' * (G' * 1)
  with Gtposeones.array_read() as gtposeones, Uq.array_read() as uq:
    for i in range(Pdim):
      sum1 += gtposeones[i]*u[i]
    for i in range(dim*Qdim):
      sum2 += uq[i]

  # Check that (1' * G * u - u' * (G' * 1)) is numerically zero
  print('1T * G * u - uT * (GT * 1) =', np.abs(sum1 - sum2))

### Advanced topics

* In the following example, we demonstrate the QR factorization of a basis matrix.
The representation is similar to LAPACK's [`dgeqrf`](https://www.netlib.org/lapack/explore-html/dd/d9a/group__double_g_ecomputational_ga3766ea903391b5cf9008132f7440ec7b.html#ga3766ea903391b5cf9008132f7440ec7b), with elementary reflectors in the lower triangular block, scaled by `tau`.

In [ ]:
qr = np.array([1, -1, 4, 1, 4, -2, 1, 4, 2, 1, -1, 0], dtype="float64")
tau = np.empty(3, dtype="float64")

qr, tau = ceed.qr_factorization(qr, tau, 4, 3)

print('qr =')
print(qr.reshape(4, 3))

print('tau =')
print(tau)

* In the following example, we demonstrate the symmetric Schur decomposition of a basis matrix

In [ ]:
A = np.array([0.19996678, 0.0745459, -0.07448852, 0.0332866,
              0.0745459, 1., 0.16666509, -0.07448852,
              -0.07448852, 0.16666509, 1., 0.0745459,
              0.0332866, -0.07448852, 0.0745459, 0.19996678], dtype="float64")

lam = ceed.symmetric_schur_decomposition(A, 4)

print("Q =")
for i in range(4):
  for j in range(4):
    if A[j+4*i] <= 1E-14 and A[j+4*i] >= -1E-14:
       A[j+4*i] = 0
    print("%12.8f"%A[j+4*i])

print("lambda =")
for i in range(4):
  if lam[i] <= 1E-14 and lam[i] >= -1E-14:
    lam[i] = 0
  print("%12.8f"%lam[i])

* In the following example, we demonstrate the simultaneous diagonalization of a basis matrix

In [ ]:
M = np.array([0.19996678, 0.0745459, -0.07448852, 0.0332866,
              0.0745459, 1., 0.16666509, -0.07448852,
              -0.07448852, 0.16666509, 1., 0.0745459,
              0.0332866, -0.07448852, 0.0745459, 0.19996678], dtype="float64")
K = np.array([3.03344425, -3.41501767, 0.49824435, -0.11667092,
              -3.41501767, 5.83354662, -2.9167733, 0.49824435,
              0.49824435, -2.9167733, 5.83354662, -3.41501767,
              -0.11667092, 0.49824435, -3.41501767, 3.03344425], dtype="float64")

x, lam = ceed.simultaneous_diagonalization(K, M, 4)

print("x =")
for i in range(4):
  for j in range(4):
    if x[j+4*i] <= 1E-14 and x[j+4*i] >= -1E-14:
      x[j+4*i] = 0
    print("%12.8f"%x[j+4*i])

print("lambda =")
for i in range(4):
  if lam[i] <= 1E-14 and lam[i] >= -1E-14:
    lam[i] = 0
  print("%12.8f"%lam[i])